In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from flatsurf import *
import numpy as np
from matplotlib import pyplot as plt
import os
import pwlf
import os
from surface_dynamics.all import *
from Library import *
from Library import Section
import math
from time import time
import copy
from scipy import integrate
import sympy as sym
from sympy import Symbol
from sympy import solve, lambdify
import traceback
import dill
import sys

In [2]:
# list of permutations
def perms_list(n, **kwargs):
    obstructed = []
    p = D.query(nb_squares=n, **kwargs)
    for item in p:
        if not is_unobstructed(item):
            obstructed.append(item)
            for perm in item.teichmueller_curve():
                obstructed.append(perm)
    return obstructed

In [3]:
# take files from saddle.py and load them into notebook
def load_arrays_from_file(file_path):
    # Load arrays from the NumPy file
    arrays_list = np.load(file_path, allow_pickle=True)
    
    # Ensure each element in the list is a NumPy array
    arrays_list = [np.array(array) for array in arrays_list]
    
    return arrays_list

In [4]:
# 7-0, 7-3, 7-7, 7-19, 7-29, 7-41
n_squares = 7
index = 1
j = 0
dx = 0.0005

In [5]:
permutations = perms_list(n_squares)
perm = permutations[index]

vec_file = "vecs" + str(n_squares) + "-" + str(index) + ".npy"
vecs0 = load_arrays_from_file(os.path.join("vecs", vec_file))

with open(os.path.join("results", f"{n_squares} - {index}", "setup.dill"), 'rb') as f:
    loaded_data = dill.load(f)
a,c,e,g = loaded_data

df = read_df(n_squares, index, j)

with open(os.path.join("results", f"{n_squares} - {index}", f"secs - {j}.dill"), 'rb') as f:
    secs = dill.load(f)

In [9]:
vecs0 = vecs0[:500000]
len(vecs0)

500000

In [7]:
def winners1(vecs, x_vals, m0, m1, y0, dx, dx_y):
    winners = []
    possible_vecs = []
    t0 = time()
    
    dz = 1 / 100
    
    # Stack list of numpy arrays into a single 2D array
    vecs = np.hstack(vecs)  # Each vec is a column in this 2D array
    
    for a in np.arange(dz, 1, dz):
        Mab = np.array([[a, 1 - dx / y0], [0, a]])  # Matrix stays outside the inner loop
    
        # Apply the transformation to all vectors at once
        new_vecs = Mab @ vecs
    
        # Extract x and y components
        x_vals = new_vecs[0, :]
        y_vals = new_vecs[1, :]
    
        # Filter based on conditions (x > 0, y/x > 0, and x <= 1)
        valid_mask = (x_vals > 0) & (x_vals <= 1) & (y_vals / x_vals > 0)
    
        valid_x = x_vals[valid_mask]
        valid_y = y_vals[valid_mask]
        valid_vecs = vecs[:, valid_mask]  # Apply the mask to filter valid vectors
    
        if valid_x.size == 0:
            winners.append(None)
            continue
    
        # Calculate slopes
        slopes = valid_y / valid_x
    
        # Find the minimum slope and handle continuity cases
        min_slope_idx = np.argmin(slopes)
        winner_slope = slopes[min_slope_idx]
        winner = valid_vecs[:, min_slope_idx]
    
        # Handle continuity by finding the smallest vector if slopes are close
        for i, slope in enumerate(slopes):
            if np.abs(slope - winner_slope) <= dx / 1000:
                if valid_vecs[0, i] < winner[0] or valid_vecs[1, i] < winner[1]:
                    winner = valid_vecs[:, i]
    
        winners.append(winner.reshape(2,1))
        
    winners2 = []
    for winner in winners:
        try:
            if winner == None:
                continue
        except:
            winners2.append(winner)
    possible_vecs1 = np.unique(winners2, axis=0)
    for item in possible_vecs1:
        possible_vecs.append(item)
    tf = time()
    print(possible_vecs)
    print(tf-t0)

In [8]:
def winners0(vecs, x_vals, m0, m1, y0, dx, dx_y):
    possible_vecs = []
    winners = []
    t0 = time()
    # top edge
    dz = 1/100
    for a in np.arange(dz, 1, dz):
        check = 0
        winner_slope = None
        winner = None
        Mab = np.array([[a, 1-dx/y0], [0, a]])
        for vec in vecs:
            new = Mab@vec
            if float(new[0][0]) == 0:
                continue
            x = float(new[0][0])
            y = float(new[1][0])
            if y/x <= 0:
                continue
            if x <= 1 and x > 0:
                if winner_slope == None:
                    winner_slope = y/x
                    winner = vec
                    continue
        #if you have two potential winners like (m,n) and 2*(m,n), make (m,n) winner for continuity and plotting purposes
                elif abs(y/x - winner_slope) <= dx/1000:
                    if vec[0][0] < winner[0][0] or vec[1][0] < winner[1][0]:
                        winner = vec
                        continue
                elif y/x < winner_slope:
                    winner_slope = y/x
                    winner = vec
                    continue
        winners.append(winner)

    winners2 = []
    for winner in winners:
        try:
            if winner == None:
                continue
        except:
            winners2.append(winner)
    possible_vecs1 = np.unique(winners2, axis=0)
    for item in possible_vecs1:
        possible_vecs.append(item)
    tf = time()
    print(possible_vecs)
    print(tf-t0)

In [ ]:
for j in range(len(a[0])):
    #change back to False in future
    improved = True
    if j == 0:
        improved = True
        
    for i in range(len(a)):

        #get dimensions of section 
        vecs, x_vals, m0, m1, x0, y0, dx_y, z = setup(a[i][j], c[i][j], e[i][j], vecs0, dx, improved)
        #print(z)
        #print("i = " + str(i), "j = " + str(j))

        if float(z) <= float(1/50000):
            print("too small")
            continue

        #create a dataframe with winning vector at certain points in the section
        winners0(vecs, x_vals, m0, m1, y0, dx, dx_y)
        winners1(vecs, x_vals, m0, m1, y0, dx, dx_y)
        print("\n\n")
        break

[array([[0.],
       [1.]])]
172.71704149246216
[array([[0.],
       [1.]])]
0.7537479400634766



